<a href="https://colab.research.google.com/github/huyhoang1996vn/my-ml/blob/master/TFIDF_vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [ ]:
!pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=0f545ed06f079af81c48683dbf8a8ffcea31cad613defe4dbe30c5d15153ae96
  Stored in directory: /root/.cache/pip/wheels/5f/d4/d7/4189b07b5902ee9f3ce0dbb14909fbe8037c39d6c63ffd49c9
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
     

To read an .ods file into a pandas DataFrame, you can use the `read_excel` function and specify the engine as `odf`.

In [ ]:
!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=3068405f9535a6e37b5ab38605ff3b8762cc7166c44822bf8cdc5b4f57102492
  Stored in directory: /root/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [ ]:
# Read the .ods file into a pandas DataFrame
df = pd.read_excel('final_project.ods', engine='odf', dtype=str)

# Display the first 5 rows of the DataFrame
display(df.head())

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report", explorative=True)
profile.to_file("profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 6/6 [00:08<00:00,  1.34s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df.isna().sum()

,0
title,0
location,0
description,1
function,0
industry,0
career_level,0


In [ ]:
def extract_second_part_of_location(location):
  """
  Checks if a location string contains a comma, splits it if so, and returns the second part.

  Args:
    location: The location string.

  Returns:
    The second part of the location string if a comma is found and splitting is successful,
    otherwise returns the original location.
  """
  if isinstance(location, str) and ',' in location:
    parts = location.split(',', 1) # Split only on the first comma
    if len(parts) > 1:
      return parts[1].strip() # Return the second part and remove leading/trailing whitespace
  return location

# Apply the function to the 'location' column
df['location'] = df['location'].apply(extract_second_part_of_location)

# Display the updated DataFrame
display(df.head())

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,TX,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,WA,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,TX,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),TX,Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [ ]:
df.dropna(subset=['description'], inplace=True)
display(df.isna().sum())
display(df.head())
display(df.describe())

,0
title,0
location,0
description,0
function,0
industry,0
career_level,0


,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,TX,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,WA,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,TX,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),TX,Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


,title,location,description,function,industry,career_level
count,8073,8073,8073,8073,8073,8073
unique,6789,97,7973,19,352,6
top,Account Manager,CA,Practice Medicine in Northern California with ...,sales,Software Companies,senior_specialist_or_project_manager
freq,58,1732,5,3150,688,4337


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('career_level', axis=1), df['career_level'], test_size=0.2, random_state=42, stratify=df['career_level'])

In [ ]:
y_train.value_counts()

,count
career_level,
senior_specialist_or_project_manager,3469
manager_team_leader,2138
bereichsleiter,768
director_business_unit_leader,56
specialist,24
managing_director_small_medium_company,3


In [ ]:
y_test.value_counts()

,count
career_level,
senior_specialist_or_project_manager,868
manager_team_leader,534
bereichsleiter,192
director_business_unit_leader,14
specialist,6
managing_director_small_medium_company,1


In [ ]:
print(y_train.value_counts())
from imblearn.over_sampling import RandomOverSampler, SMOTEN
from collections import Counter
ros = RandomOverSampler(sampling_strategy={
    'director_business_unit_leader': 500,
    'specialist': 500,
    'managing_director_small_medium_company': 500,
}, random_state=42)

# ros = SMOTEN(sampling_strategy={
#     'director_business_unit_leader': 500,
#     'specialist': 500,
#     'managing_director_small_medium_company': 500,
# }, k_neighbors=2 ,random_state=42)

# Resample the dataset
X_resampled, y_resampled = ros.fit_resample(x_train, y_train)

print(f"Resampled dataset shape: {Counter(y_resampled)}")
print(y_resampled.value_counts())
x_train=X_resampled
y_train=y_resampled

career_level
senior_specialist_or_project_manager      3469
manager_team_leader                       2138
bereichsleiter                             768
director_business_unit_leader               56
specialist                                  24
managing_director_small_medium_company       3
Name: count, dtype: int64
Resampled dataset shape: Counter({'senior_specialist_or_project_manager': 3469, 'manager_team_leader': 2138, 'bereichsleiter': 768, 'specialist': 500, 'director_business_unit_leader': 500, 'managing_director_small_medium_company': 500})
career_level
senior_specialist_or_project_manager      3469
manager_team_leader                       2138
bereichsleiter                             768
specialist                                 500
director_business_unit_leader              500
managing_director_small_medium_company     500
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Create separate TfidfVectorizer instances for each text column
tfidf_title = TfidfVectorizer()
tfidf_industry = TfidfVectorizer()
tfidf_function = TfidfVectorizer()
tfidf_description = TfidfVectorizer(ngram_range=(1, 2), min_df=0.01, max_df=0.95) # Using ngram_range=(1,2) for description as before

# Create a OneHotEncoder instance for the location column
one_hot_location = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf_title', tfidf_title, 'title'),
        ('tfidf_industry', tfidf_industry, 'industry'),
        ('tfidf_function', tfidf_function, 'function'),
        ('tfidf_description', tfidf_description, 'description'),
        ('one_hot_location', one_hot_location, ['location']),
    ])

# Fit and transform the training data
x_train_processed = preprocessor.fit_transform(x_train)

# Display the shape of the processed training data


In [ ]:
print(x_train_processed.shape)

(6458, 11396)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile

reg=Pipeline(steps=[
    ('preprocessor',preprocessor),
    # ('select_k_best',SelectKBest(score_func=chi2, k=1200)),
    ('select_percent',SelectPercentile(score_func=chi2, percentile=20)),
     ('regressor', RandomForestClassifier())
])
reg.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf_title',
                                                  TfidfVectorizer(), 'title'),
                                                 ('tfidf_industry',
                                                  TfidfVectorizer(),
                                                  'industry'),
                                                 ('tfidf_function',
                                                  TfidfVectorizer(),
                                                  'function'),
                                                 ('tfidf_description',
                                                  TfidfVectorizer(max_df=0.95,
                                                                  min_df=0.01,
                                                                  ngram_range=(1,
                                                                               2)),
                                                  'description'),
                                                 ('one_hot_location',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['location'])])),
                ('select_percent',
                 SelectPercentile(percentile=20,
                                  score_func=<function chi2 at 0x7c6a6cfd7560>)),
                ('regressor', RandomForestClassifier())])

In [ ]:
y_predict = reg.predict(x_test)

In [ ]:
y_test.value_counts()

,count
career_level,
senior_specialist_or_project_manager,868
manager_team_leader,534
bereichsleiter,192
director_business_unit_leader,14
specialist,6
managing_director_small_medium_company,1


In [ ]:
from sklearn.metrics import classification_report
report=classification_report(y_test, y_predict)
print(report)

                                        precision    recall  f1-score   support

                        bereichsleiter       0.65      0.08      0.14       192
         director_business_unit_leader       1.00      0.36      0.53        14
                   manager_team_leader       0.64      0.74      0.68       534
managing_director_small_medium_company       0.00      0.00      0.00         1
  senior_specialist_or_project_manager       0.83      0.93      0.88       868
                            specialist       1.00      0.17      0.29         6

                              accuracy                           0.76      1615
                             macro avg       0.69      0.38      0.42      1615
                          weighted avg       0.75      0.76      0.72      1615



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_predict)
# precision = precision_score(y_test, y_predict, pos_label='yes')
# recall = recall_score(y_test, y_predict, pos_label='yes')
# f1 = f1_score(y_test, y_predict, pos_label='yes')

print(f"Accuracy: {accuracy:.2f}")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1-score: {f1:.2f}")

Accuracy: 0.73


In [ ]:
from sklearn.model_selection import GridSearchCV
simple_param_grid = {
    'select_percent__percentile': [5, 10, 15, 20]
}

# Create GridSearchCV object
grid_search = GridSearchCV(
    estimator=reg,
    param_grid=simple_param_grid,  # or simple_param_grid for fewer combinations
    cv=5,                   # 5-fold cross-validation
    scoring='recall_weighted',  # or 'r2', 'neg_mean_absolute_error'
    n_jobs=-1,              # Use all available CPUs
    verbose=1               # Show progress
)
# Fit the grid search
grid_search.fit(x_train, y_train)
grid_y_predict = grid_search.predict(x_test)
print(f"Best best_estimator: {grid_search.best_estimator_}")

# Print the best hyperparameters found
print(f"Best hyperparameters: {grid_search.best_params_}")

# Print the best accuracy achieved
print(f"Best cross-validated: {grid_search.best_score_:.4f}")
from sklearn.metrics import classification_report
report1=classification_report(y_test, grid_y_predict)
print(report1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best best_estimator: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf_title',
                                                  TfidfVectorizer(), 'title'),
                                                 ('tfidf_industry',
                                                  TfidfVectorizer(),
                                                  'industry'),
                                                 ('tfidf_function',
                                                  TfidfVectorizer(),
                                                  'function'),
                                                 ('tfidf_description',
                                                  TfidfVectorizer(max_df=0.95,
                                                                  min_df=0.01,
                                                                  ngram_range=(1,
                               

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Access the fitted preprocessor and feature selector from the pipeline
fitted_preprocessor = reg.named_steps['preprocessor']
fitted_selector = reg.named_steps['select_percent']
fitted_classifier = reg.named_steps['regressor']

# Get the feature names after preprocessing
preprocessed_feature_names = []

# Get feature names for TF-IDF transformed columns
for name, transformer, original_column in fitted_preprocessor.transformers_[:-1]: # Exclude the last transformer which is OneHotEncoder
    if hasattr(transformer, 'get_feature_names_out'):
         preprocessed_feature_names.extend(transformer.get_feature_names_out([original_column]))
    else:
         # Fallback for older versions or transformers without get_feature_names_out
         preprocessed_feature_names.extend([f"{original_column}_{i}" for i in range(transformer.transform(x_train[[original_column]]).shape[1])])


# Get feature names for OneHotEncoded column
name, transformer, original_columns = fitted_preprocessor.transformers_[-1] # Get the last transformer
if hasattr(transformer, 'get_feature_names_out'):
     preprocessed_feature_names.extend(transformer.get_feature_names_out(original_columns))
else:
     # Fallback for older versions or transformers without get_feature_names_out
     preprocessed_feature_names.extend([f"{original_columns[0]}_{i}" for i in range(transformer.transform(x_train[original_columns]).shape[1])])


# Get the mask of selected features from SelectPercentile
selected_features_mask = fitted_selector.get_support()

# Get the names of the selected features
selected_feature_names = [preprocessed_feature_names[i] for i in range(len(preprocessed_feature_names)) if selected_features_mask[i]]

# Get the feature importances from the Random Forest model
feature_importances = fitted_classifier.feature_importances_

# Create a pandas Series for better visualization
importance_series = pd.Series(feature_importances, index=selected_feature_names)

# Sort the features by importance and display the top N
top_n = 20  # You can adjust this number
print(f"Top {top_n} most important features:")
display(importance_series.sort_values(ascending=False).head(top_n))

NameError: name 'reg' is not defined